## Mounting the drive and unziping the Data Folder

In [6]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

zip_path = '/content/drive/MyDrive/processed_data.zip'  # your ZIP file
extract_path = '/content/processed_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Check contents
!ls /content/processed_data


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
processed_data


## Moving all emotion folders to root

In [11]:
import shutil
import os

src_inner = '/content/processed_data/processed_data'
dst_root = '/content/processed_data'

# Move all emotion folders to root
for item in os.listdir(src_inner):
    shutil.move(os.path.join(src_inner, item), dst_root)

# Remove now-empty folder
os.rmdir(src_inner)

# Verify
!ls /content/processed_data


angry  disgust	fear  happy  neutral  sad  surprise  train  val


## Splitting the Data into train and val

In [12]:
from sklearn.model_selection import train_test_split
from glob import glob
import shutil
import os

data_dir = '/content/processed_data'
train_dir = os.path.join(data_dir, 'train')
val_dir   = os.path.join(data_dir, 'val')

# Create train/val folders if they don't exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Loop through emotion folders
for emotion in ['angry','disgust','fear','happy','neutral','sad','surprise']:
    emotion_path = os.path.join(data_dir, emotion)
    images = glob(os.path.join(emotion_path, '*.jpg'))
    if not images:
        continue

    train_imgs, val_imgs = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_dir, emotion), exist_ok=True)
    os.makedirs(os.path.join(val_dir, emotion), exist_ok=True)

    # Copy images
    for img in train_imgs:
        shutil.copy(img, os.path.join(train_dir, emotion))
    for img in val_imgs:
        shutil.copy(img, os.path.join(val_dir, emotion))

print("✅ Dataset properly split into 'train' and 'val' folders!")


✅ Dataset properly split into 'train' and 'val' folders!


## To be Show of my Directory Where my Data is

In [13]:
!ls /content/processed_data/train
!ls /content/processed_data/val


angry  disgust	fear  happy  neutral  sad  surprise
angry  disgust	fear  happy  neutral  sad  surprise


## Calculate class weights based on the frequency of classes

In [14]:
import torch
import torch.nn as nn

# Calculate class weights based on the frequency of classes
class_counts = [len(os.listdir(f'/content/processed_data/train/{cls}')) for cls in train_dataset.classes]
class_weights = [sum(class_counts) / count for count in class_counts]
class_weights = torch.tensor(class_weights).float().cuda()

# Use class weights in your loss function
criterion = nn.CrossEntropyLoss(weight=class_weights)


## Data Preprocessing

In [15]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(10),  # Random rotation for better generalization
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),  # Color jitter
        transforms.ToTensor(),
        transforms.Normalize([0.0911, -0.0043, -0.0403], [0.3940, 0.3806, 0.3741])  # Your calculated mean and std
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.0911, -0.0043, -0.0403], [0.3940, 0.3806, 0.3741])  # Same normalization for validation
    ])
}


train_dataset = datasets.ImageFolder('/content/processed_data/train', transform=data_transforms['train'])
val_dataset   = datasets.ImageFolder('/content/processed_data/val', transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)

print("✅ Dataset loaded!")
print("Training images:", len(train_dataset))
print("Validation images:", len(val_dataset))
print("Classes:", train_dataset.classes)


✅ Dataset loaded!
Training images: 47772
Validation images: 17907
Classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


## Training the model

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from facenet_pytorch import InceptionResnetV1
from tqdm import tqdm

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load pretrained model
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=7).to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Epoch {epoch+1}/{epochs} — Loss: {epoch_loss:.4f}, Val Acc: {val_acc:.4f}")

print("✅ Training completed!")


Using device: cuda


Epoch 1/10: 100%|██████████| 1495/1495 [11:22<00:00,  2.19it/s]


Epoch 1/10 — Loss: 0.9863, Val Acc: 0.6732


Epoch 2/10: 100%|██████████| 1495/1495 [11:21<00:00,  2.20it/s]


Epoch 2/10 — Loss: 0.8163, Val Acc: 0.7315


...

Epoch 8/10 — Loss: 0.4919, Val Acc: 0.8661


Epoch 9/10: 100%|██████████| 1495/1495 [11:16<00:00,  2.21it/s]


Epoch 9/10 — Loss: 0.4405, Val Acc: 0.8703


Epoch 10/10: 100%|██████████| 1495/1495 [11:17<00:00,  2.21it/s]


Epoch 10/10 — Loss: 0.4035, Val Acc: 0.9003
✅ Training completed!


## Saving the model

In [8]:
import torch

# Make sure 'model' is defined in this session (your trained InceptionResnetV1)
save_path = '/content/drive/MyDrive/emotion_model2.pth'  # You can change folder/name if you want
torch.save(model.state_dict(), save_path)

print(f"✅ Model saved successfully at {save_path}")

✅ Model saved successfully at /content/drive/MyDrive/emotion_model2.pth


## Reloading the model again from the Directory

In [9]:
import torch
from facenet_pytorch import InceptionResnetV1

# Define the model architecture
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=7)

# Load the trained weights
model.load_state_dict(torch.load('/content/drive/MyDrive/emotion_model2.pth'))
model.eval()  # Set to evaluation mode

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("✅ Model loaded successfully!")


✅ Model loaded successfully!


## Testing the model on the Validation Data

In [11]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from facenet_pytorch import InceptionResnetV1
from tqdm import tqdm

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Paths (update if needed)
val_dir = "/content/processed_data/val"
model_path = "/content/drive/MyDrive/emotion_model2.pth"

# Define same transforms used during training
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.0911, -0.0043, -0.0403], [0.3940, 0.3806, 0.3741])
])

# Load validation dataset
val_dataset = datasets.ImageFolder(val_dir, transform=data_transforms)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Load model
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=7)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()
model = model.to(device)

# Validation loop
correct = 0
total = 0
running_loss = 0.0
criterion = torch.nn.CrossEntropyLoss()

with torch.no_grad():
    for images, labels in tqdm(val_loader, desc="Validating"):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        running_loss += loss.item() * images.size(0)

        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_acc = correct / total
val_loss = running_loss / len(val_loader.dataset)

print(f"\n✅ Validation completed!")
print(f"Validation Accuracy: {val_acc:.4f}")
print(f"Validation Loss: {val_loss:.4f}")


Using device: cuda


Validating: 100%|██████████| 561/561 [01:16<00:00,  7.37it/s]


✅ Validation completed!
Validation Accuracy: 0.9003
Validation Loss: 0.2925


## USING WEBCAM TO COLLECT INPUT IT IS NOT WORKING ON COLAB ONLY ON YOUR LOCAL MACHINE

In [12]:
import cv2
import torch
from facenet_pytorch import InceptionResnetV1
from torchvision import transforms
from PIL import Image

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InceptionResnetV1(pretrained='vggface2', classify=True, num_classes=7)
model.load_state_dict(torch.load('/content/drive/MyDrive/emotion_model2.pth'))  # path to your saved model
model.eval()
model = model.to(device)

# Define transform
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.0911, -0.0043, -0.0403], [0.3940, 0.3806, 0.3741])
])

# Class labels
classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Start webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert frame to PIL image
    img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    img_tensor = transform(img).unsqueeze(0).to(device)

    # Predict emotion
    with torch.no_grad():
        outputs = model(img_tensor)
        _, pred = torch.max(outputs, 1)
        emotion = classes[pred.item()]

    # Display the result
    cv2.putText(frame, f"Emotion: {emotion}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)
    cv2.imshow("Webcam Emotion Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
